In [ ]:
#Harris' version with ID fixed #turn page not yet work. Fix it!
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
#want to add options to make it headless here?? - Steven 
import time
import pandas as pd
import random
import threading
from concurrent.futures import ThreadPoolExecutor
from datetime import datetime 

# 鞋碼轉換對照表，包含 UK5 至 UK10
size_conversion = {
    "UK5": {"US": "6.5", "EU": "38"},
    "UK5.5": {"US": "7", "EU": "38.7"},
    "UK6": {"US": "7.5", "EU": "39.3"},
    "UK6.5": {"US": "8", "EU": "40"},
    "UK7": {"US": "8.5", "EU": "40.7"},
    "UK7.5": {"US": "9", "EU": "41.3"},
    "UK8": {"US": "9.5", "EU": "42"},
    "UK8.5": {"US": "10", "EU": "42.7"},
    "UK9": {"US": "10.5", "EU": "43.3"},
    "UK9.5": {"US": "11", "EU": "44"},
    "UK10": {"US": "11.5", "EU": "44.7"}
}

# US 轉 UK 的反向對照表
us_to_uk = {v["US"]: k.replace("UK", "") for k, v in size_conversion.items()}

# 定義鞋碼範圍，包含 UK5 至 UK10
uk_sizes = ["5", "5.5", "6", "6.5", "7", "7.5", "8", "8.5", "9", "9.5", "10"]

# 定義鞋碼欄位名稱映射
size_column_names = {
    "5": "uk_size_5",
    "5.5": "uk_size_5_5",
    "6": "uk_size_6",
    "6.5": "uk_size_6_5",
    "7": "uk_size_7",
    "7.5": "uk_size_7_5",
    "8": "uk_size_8",
    "8.5": "uk_size_8_5",
    "9": "uk_size_9",
    "9.5": "uk_size_9_5",
    "10": "uk_size_10"
}

# 定義折扣加權值映射（統一為小寫）
discount_weights = {
    "vip 10% off": 90,
    "vip 15% off": 85,
    "10% off": 90,
    "buy 1 get 10% off": 90,
    "buy 2 get 20% off": 80,
    "buy 1 get 15% off": 85,
    "buy 3 get 30% off": 70,
    "buy 2 get 10% off": 90,
    "no discount": 100
}

# 計算折扣加權值的函數（忽略大小寫）
def calculate_discount_weight(discount_text):
    # 將折扣文字轉為小寫進行匹配
    discount_text_lower = discount_text.lower()
    if discount_text_lower in discount_weights:
        return discount_weights[discount_text_lower]
    return 1.0

# 鎖，用於保護共享資源
lock = threading.Lock()

# 每個執行緒的抓取函數
def fetch_details(index, product_name, href, size_status_list, sku_list):
    driver = webdriver.Chrome()
    try:
        print(f'執行緒 {threading.current_thread().name} 正在處理第 {index+1} 個產品: {product_name}')
        driver.get(href)
        print(f'執行緒 {threading.current_thread().name} 已進入產品 {index+1} 的詳情頁面: {product_name}')
        time.sleep(random.uniform(3, 5))

        try:
            popup_close = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "Close") or contains(text(), "OK") or @aria-label="Close"]'))
            )
            driver.execute_script("arguments[0].click();", popup_close)
            print(f'執行緒 {threading.current_thread().name} 已關閉詳情頁面彈窗')
        except:
            print(f'執行緒 {threading.current_thread().name} 無詳情頁面彈窗或無法關閉')

        # 抓取鞋碼並轉換為數字
        size_status = {size_column_names[uk_size]: 0 for uk_size in uk_sizes}
        try:
            size_elements = WebDriverWait(driver, 15).until(
                EC.presence_of_all_elements_located((By.XPATH, '//div[contains(@class, "swatch-option text") and (contains(@data-option-label, "UK") or contains(@data-option-label, "US"))]'))
            )
            if size_elements:
                for size in size_elements:
                    size_label = size.get_attribute("data-option-label")
                    if not size_label:
                        continue
                    uk_size = None
                    if "UK" in size_label:
                        uk_size = size_label.replace("UK", "").strip()
                    elif "US" in size_label:
                        us_size = size_label.replace("US", "").strip()
                        uk_size = us_to_uk.get(us_size)
                    if uk_size and uk_size in uk_sizes:
                        style = size.get_attribute("style")
                        status = 2 if style == "" else 1  # 有貨: 2, 缺貨: 1
                        size_status[size_column_names[uk_size]] = status
                        print(f'執行緒 {threading.current_thread().name} 鞋碼 UK{uk_size}: {status}')
            else:
                print(f'執行緒 {threading.current_thread().name} 產品 {index+1} 無鞋碼')
        except:
            print(f'執行緒 {threading.current_thread().name} 產品 {index+1} 抓取失敗：鞋碼元素未找到')
            size_status = {size_column_names[uk_size]: 0 for uk_size in uk_sizes}

        # 抓取 SKU
        try:
            sku_element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//div[@class="value" and @itemprop="sku"]'))
            )
            sku = sku_element.text.strip()
            print(f'執行緒 {threading.current_thread().name} 產品 {index+1} SKU: {sku}')
        except:
            sku = "無 SKU"
            print(f'執行緒 {threading.current_thread().name} 產品 {index+1} 無 SKU')

        with lock:
            size_status_list[index] = size_status
            sku_list[index] = sku

    except Exception as e:
        print(f'執行緒 {threading.current_thread().name} 產品 {index+1} 抓取失敗: {e}')
        with lock:
            size_status_list[index] = {size_column_names[uk_size]: 0 for uk_size in uk_sizes}
            sku_list[index] = "抓取失敗"
    finally:
        driver.quit()

# 主程式 #add all the urls #COMMENT ALL NOT USED URL
driver = webdriver.Chrome()
# url = 'https://marathonsports.hkstore.com/marathon_en_us/women/footwear'
# url = 'https://marathonsports.hkstore.com/marathon_en_us/men/footwear'
url = 'https://catalog.hkstore.com/catalog_en_us/women/footwear'
# url = 'https://catalog.hkstore.com/catalog_en_us/men/footwear'



try:
    driver.get(url)
    time.sleep(5)
except:
    print(f'failed to open {url}')

try:
    cookie_button = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "Accept") or contains(text(), "Agree") or contains(text(), "OK")]'))
    )
    driver.execute_script("arguments[0].click();", cookie_button)
    print("已關閉彈窗")
except:
    print("無彈窗或無法關閉")

# 初始化清單儲存資料
brand_list = []
product_name_list = []
gender_list = []
type_list = []
price_list = []
discount1_list = []
discount2_list = []
discount3_list = []
discount1_weighted_list = []
discount2_weighted_list = []
discount3_weighted_list = []
sku_list = []
href_list = []

# 一次性滾動頁面，抓取所有產品資訊
previous_count = 0
max_attempts = 5
attempt = 0

while attempt < max_attempts:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)
    result_list = driver.find_elements(By.XPATH, '//div[@class="product-item-info"]')
    current_count = len(result_list)
    print(f'滾動後找到 {current_count} 個產品')
    if current_count == previous_count:
        break
    previous_count = current_count
    attempt += 1

# 抓取品牌
brand_elements = driver.find_elements(By.XPATH, '//div[@class="product-item-brand"]')
print(f'找到 {len(brand_elements)} 個品牌')
for brand in brand_elements:
    print(brand.text)
    brand_list.append(brand.text)

# 抓取產品名稱
product_name_elements = driver.find_elements(By.XPATH, '//strong[@class="product name product-item-name"]')
print(f'找到 {len(product_name_elements)} 個產品名稱')
for name in product_name_elements:
    print(name.text)
    product_name_list.append(name.text)

# 抓取 Gender 和 Type
gender_type_elements = driver.find_elements(By.XPATH, '//div[@class="product-item-gender-type"]')
print(f'找到 {len(gender_type_elements)} 個性別類型')
for gender_type in gender_type_elements:
    text = gender_type.text.strip()
    print(f'原始 GenderType: {text}')
    if text:
        parts = text.split(" ")
        gender = parts[0]
        type_ = " ".join(parts[1:]) if len(parts) > 1 else "未知類型"
    else:
        gender = "未知性別"
        type_ = "未知類型"
    print(f'Gender: {gender}, Type: {type_}')
    gender_list.append(gender)
    type_list.append(type_)

# 抓取價格
price_elements = driver.find_elements(By.XPATH, '//div[@class="price-box price-final_price"]')
print(f'找到 {len(price_elements)} 個價格')
for price in price_elements:
    print(price.text)
    price_text = price.text.replace("HK$", "").replace(",", "")
    try:
        price_value = float(price_text)
    except ValueError:
        price_value = 0.0
    price_list.append(price_value)

# 抓取折扣並拆分
discount1_list = []
discount2_list = []
discount3_list = []
discount1_weighted_list = []
discount2_weighted_list = []
discount3_weighted_list = []

for item in result_list:
    left_discount = item.find_elements(By.XPATH, './/div[contains(@class, "cataloglabel-bottom-left")]//span[contains(text(), "Off") or contains(text(), "Buy")]')
    right_discount = item.find_elements(By.XPATH, './/div[contains(@class, "cataloglabel-botton-right")]//span[contains(text(), "Off") or contains(text(), "Buy")]')
    
    # 抓取左折扣
    left_text = left_discount[0].text.strip() if left_discount else ""
    left_discounts = []
    if left_text and left_text not in ["無左折扣", ""]:
        left_discounts = [d.strip() for d in left_text.split("|")]
    
    # 抓取右折扣
    right_text = right_discount[0].text.strip() if right_discount else ""
    right_discounts = []
    if right_text and right_text not in ["無右折扣", ""]:
        right_discounts = [d.strip() for d in right_text.split("|")]
    
    # 合併所有有效的折扣
    all_discounts = left_discounts + right_discounts
    # 如果沒有折扣，設為 ["No Discount"]
    if not all_discounts:
        all_discounts = ["No Discount"]
    # 最多取前三個折扣
    all_discounts = all_discounts[:3]
    # 補齊到三個折扣
    while len(all_discounts) < 3:
        all_discounts.append("No Discount")
    
    # 分配到三個欄位
    discount1_list.append(all_discounts[0])
    discount2_list.append(all_discounts[1])
    discount3_list.append(all_discounts[2])
    
    # 計算加權值
    discount1_weighted_list.append(calculate_discount_weight(all_discounts[0]))
    discount2_weighted_list.append(calculate_discount_weight(all_discounts[1]))
    discount3_weighted_list.append(calculate_discount_weight(all_discounts[2]))
    
    print(f'折扣1: {all_discounts[0]} (加權值: {discount1_weighted_list[-1]}) | 折扣2: {all_discounts[1]} (加權值: {discount2_weighted_list[-1]}) | 折扣3: {all_discounts[2]} (加權值: {discount3_weighted_list[-1]})')

# 抓取 href
for item in result_list:
    try:
        link = item.find_element(By.XPATH, './/a[.//img[@class="product-image-photo"]]')
        href = link.get_attribute("href")
        href_list.append(href)
    except:
        href_list.append("無連結")

# 關閉主頁面的瀏覽器
driver.quit()

# 初始化鞋碼狀態和 SKU 清單（預先分配空間，避免執行緒衝突）
total_products = len(product_name_list)
size_status_list = [None] * total_products
sku_list = [None] * total_products

# 使用多執行緒抓取詳情頁面資料
max_workers = 4
with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = []
    for index in range(total_products):
        if href_list[index] == "無連結":
            print(f'產品 {index+1} 無連結，跳過')
            size_status_list[index] = {size_column_names[uk_size]: 0 for uk_size in uk_sizes}
            sku_list[index] = "無連結"
            continue
        future = executor.submit(fetch_details, index, product_name_list[index], href_list[index], size_status_list, sku_list)
        futures.append(future)

    for future in futures:
        future.result()

# 調整清單長度（確保所有欄位行數一致）
max_length = max(len(brand_list), len(product_name_list), len(gender_list), len(type_list), len(price_list), len(discount1_list), len(discount2_list), len(discount3_list), len(sku_list))
for lst in [brand_list, product_name_list, gender_list, type_list, price_list, discount1_list, discount2_list, discount3_list, discount1_weighted_list, discount2_weighted_list, discount3_weighted_list, sku_list]:
    while len(lst) < max_length:
        lst.append("N/A")

# 根據 url 判斷 website 的值
if url.startswith('https://marathonsports.hkstore.com/marathon_en_us/'):
    website_value = 0
    website_code = "01"  # 馬拉松
elif url.startswith('https://catalog.hkstore.com/catalog_en_us/'):
    website_value = 1
    website_code = "02"  # Catalog
else:
    website_value = 2
    website_code = "00"  # 其他網站

# 獲取當前日期，格式為 YYMMDD（例如 250414）
date_str = datetime.now().strftime("%y%m%d")

# 生成 scrape_id：網站代碼 + 日期 + 編號（兩位數）
scrape_ids = [f"{website_code}{date_str}{str(i).zfill(2)}" for i in range(len(product_name_list))]

# 整理成表格
data = {
    'scrape_id': scrape_ids,  # 使用新的 scrape_id 格式
    'shoe_brand': brand_list,
    'shoe_name': product_name_list,
    'shoe_gender': gender_list,
    'shoe_type': type_list,
    'price': price_list,
    'discount_1_text': discount1_list,
    'discount_1_weighted': discount1_weighted_list,
    'discount_2_text': discount2_list,
    'discount_2_weighted': discount2_weighted_list,
    'discount_3_text': discount3_list,
    'discount_3_weighted': discount3_weighted_list,
    'shoe_sku': sku_list,
    'website': [website_value] * len(product_name_list),
    'page_ranking': list(range(len(product_name_list)))
}

# 將鞋碼狀態加入表格
for uk_size in uk_sizes:
    column_name = size_column_names[uk_size]
    data[column_name] = [status[column_name] for status in size_status_list]

df = pd.DataFrame(data)

# 添加抓取日期
scrape_date = datetime.now().strftime("%Y-%m-%d")
df['scrape_time'] = scrape_date

# 分成兩個表格並輸出為 CSV
date_prefix = datetime.now().strftime("%Y%m%d")  # 格式為 YYYYMMDD，例如 20250413

# 第一個表格：marathon_shoes.csv
df_shoes = df[['shoe_sku', 'shoe_name', 'shoe_brand', 'shoe_type', 'shoe_gender']]
# df_shoes.to_csv(f'{date_prefix}_marathon_shoes_women.csv', index=False, encoding='utf-8-sig')
# print(f"已將第一個表格儲存為 '{date_prefix}_marathon_shoes_women.csv'")

# 第二個表格：marathon_scrape_data.csv
df_scrape_data = df[['scrape_id', 'shoe_sku', 'price', 
                     'discount_1_text', 'discount_1_weighted', 
                     'discount_2_text', 'discount_2_weighted', 
                     'discount_3_text', 'discount_3_weighted',
                     'uk_size_5', 'uk_size_5_5', 'uk_size_6', 'uk_size_6_5', 'uk_size_7',
                     'uk_size_7_5', 'uk_size_8', 'uk_size_8_5', 'uk_size_9', 'uk_size_9_5', 'uk_size_10',
                     'page_ranking', 'website', 'scrape_time']]
# df_scrape_data.to_csv(f'{date_prefix}_marathon_scrape_data_women.csv', index=False, encoding='utf-8-sig')
# print(f"已將第二個表格儲存為 '{date_prefix}_marathon_scrape_data_women.csv'")

# 顯示兩個表格
print("\n第一個表格 (marathon_shoes_women):")
display(df_shoes)
print("\n第二個表格 (marathon_scrape_data_women):")
display(df_scrape_data)

無彈窗或無法關閉
滾動後找到 24 個產品
滾動後找到 24 個產品
找到 24 個品牌
Keen
Keen
Keen
UGG
UGG
Keen
Merrell
Hoka
UGG
Keen
Keen
UGG
Stare
Hoka
Keen
Hoka
Hoka
Merrell
Merrell
Hoka
Hoka
Hoka
Hoka
UGG
找到 24 個產品名稱
Zerraport II
Shanti
Howser Iii Slide
Classic Ultra Mini Boot
Tasman Slip-On
Hood Clog
Hydro Moc
Ora Recovery Slide
CA1
Newport H2
Newport H2
Classic Mini Platform
MULE
Ora Luxe
Jasper
Kawana
Challenger Atr 7
Hut Ultra Wrap
Hut Ultra Wrap
Anacapa Breeze Low
Bondi 8
Bondi 8
Mafate Speed 2
Funkette
找到 24 個性別類型
原始 GenderType: Women Sandals
Gender: Women, Type: Sandals
原始 GenderType: Women Slippers
Gender: Women, Type: Slippers
原始 GenderType: Women Slip On
Gender: Women, Type: Slip On
原始 GenderType: Women Boots
Gender: Women, Type: Boots
原始 GenderType: Women Slip on
Gender: Women, Type: Slip on
原始 GenderType: Women slip on
Gender: Women, Type: slip on
原始 GenderType: Women Sandals
Gender: Women, Type: Sandals
原始 GenderType: Unisex Sandals
Gender: Unisex, Type: Sandals
原始 GenderType: Women Sneakers
Gender: Women, 

,shoe_sku,shoe_name,shoe_brand,shoe_type,shoe_gender
0,1KN1022500,Zerraport II,Keen,Sandals,Women
1,1KN1026263,Shanti,Keen,Slippers,Women
2,1KN1026649,Howser Iii Slide,Keen,Slip On,Women
3,1UG1116109CHE,Classic Ultra Mini Boot,UGG,Boots,Women
4,1UG5955CHE,Tasman Slip-On,UGG,Slip on,Women
5,1KN1026802,Hood Clog,Keen,slip on,Women
6,1MLJ005558,Hydro Moc,Merrell,Sandals,Women
7,1HK1135061BBLC,Ora Recovery Slide,Hoka,Sandals,Unisex
8,1UG1136845SSMK,CA1,UGG,Sneakers,Women
9,1KN1027353,Newport H2,Keen,Sandals,Women



第二個表格 (marathon_scrape_data_women):


,scrape_id,shoe_sku,price,discount_1_text,discount_1_weighted,discount_2_text,discount_2_weighted,discount_3_text,discount_3_weighted,uk_size_5,...,uk_size_7,uk_size_7_5,uk_size_8,uk_size_8_5,uk_size_9,uk_size_9_5,uk_size_10,page_ranking,website,scrape_time
0,0225041400,1KN1022500,0.0,No Discount,100,No Discount,100,No Discount,100,1,...,1,1,0,0,0,0,0,0,1,2025-04-14
1,0225041401,1KN1026263,590.0,VIP 10% Off,90,No Discount,100,No Discount,100,0,...,0,2,0,0,0,0,0,1,1,2025-04-14
2,0225041402,1KN1026649,0.0,No Discount,100,No Discount,100,No Discount,100,1,...,1,1,1,0,0,0,0,2,1,2025-04-14
3,0225041403,1UG1116109CHE,1490.0,10% Off,90,No Discount,100,No Discount,100,0,...,0,2,0,1,0,1,0,3,1,2025-04-14
4,0225041404,1UG5955CHE,990.0,10% Off,90,No Discount,100,No Discount,100,1,...,1,1,1,1,1,1,0,4,1,2025-04-14
5,0225041405,1KN1026802,0.0,No Discount,100,No Discount,100,No Discount,100,2,...,2,2,1,0,0,0,0,5,1,2025-04-14
6,0225041406,1MLJ005558,0.0,No Discount,100,No Discount,100,No Discount,100,0,...,0,2,0,0,0,0,0,6,1,2025-04-14
7,0225041407,1HK1135061BBLC,0.0,No Discount,100,No Discount,100,No Discount,100,0,...,0,1,0,1,0,0,0,7,1,2025-04-14
8,0225041408,1UG1136845SSMK,0.0,No Discount,100,No Discount,100,No Discount,100,0,...,0,1,0,0,0,0,0,8,1,2025-04-14
9,0225041409,1KN1027353,950.0,VIP 10% Off,90,No Discount,100,No Discount,100,2,...,1,0,1,0,0,0,0,9,1,2025-04-14


In [10]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import random
import threading
from concurrent.futures import ThreadPoolExecutor
from datetime import datetime 

# 鞋碼轉換對照表，包含 UK5 至 UK10
size_conversion = {
    "UK5": {"US": "6.5", "EU": "38"},
    "UK5.5": {"US": "7", "EU": "38.7"},
    "UK6": {"US": "7.5", "EU": "39.3"},
    "UK6.5": {"US": "8", "EU": "40"},
    "UK7": {"US": "8.5", "EU": "40.7"},
    "UK7.5": {"US": "9", "EU": "41.3"},
    "UK8": {"US": "9.5", "EU": "42"},
    "UK8.5": {"US": "10", "EU": "42.7"},
    "UK9": {"US": "10.5", "EU": "43.3"},
    "UK9.5": {"US": "11", "EU": "44"},
    "UK10": {"US": "11.5", "EU": "44.7"}
}

# US 轉 UK 的反向對照表
us_to_uk = {v["US"]: k.replace("UK", "") for k, v in size_conversion.items()}

# 定義鞋碼範圍，包含 UK5 至 UK10
uk_sizes = ["5", "5.5", "6", "6.5", "7", "7.5", "8", "8.5", "9", "9.5", "10"]

# 定義鞋碼欄位名稱映射
size_column_names = {
    "5": "uk_size_5",
    "5.5": "uk_size_5_5",
    "6": "uk_size_6",
    "6.5": "uk_size_6_5",
    "7": "uk_size_7",
    "7.5": "uk_size_7_5",
    "8": "uk_size_8",
    "8.5": "uk_size_8_5",
    "9": "uk_size_9",
    "9.5": "uk_size_9_5",
    "10": "uk_size_10"
}

# 定義折扣加權值映射（統一為小寫）
discount_weights = {
    "vip 10% off": 90,
    "vip 15% off": 85,
    "10% off": 90,
    "buy 1 get 10% off": 90,
    "buy 2 get 20% off": 80,
    "buy 1 get 15% off": 85,
    "buy 3 get 30% off": 70,
    "buy 2 get 10% off": 90,
    "no discount": 100
}

# 計算折扣加權值的函數（忽略大小寫）
def calculate_discount_weight(discount_text):
    discount_text_lower = discount_text.lower()
    if discount_text_lower in discount_weights:
        return discount_weights[discount_text_lower]
    return 1.0

# 鎖，用於保護共享資源
lock = threading.Lock()

# 每個執行緒的抓取函數（抓取內頁資料）
def fetch_details(index, product_name, href, size_status_list, sku_list, global_index):
    driver = webdriver.Chrome()
    try:
        print(f'執行緒 {threading.current_thread().name} 正在處理第 {global_index+1} 個產品: {product_name}')
        driver.get(href)
        print(f'執行緒 {threading.current_thread().name} 已進入產品 {global_index+1} 的詳情頁面: {product_name}')
        time.sleep(random.uniform(3, 5))

        try:
            popup_close = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "Close") or contains(text(), "OK") or @aria-label="Close"]'))
            )
            driver.execute_script("arguments[0].click();", popup_close)
            print(f'執行緒 {threading.current_thread().name} 已關閉詳情頁面彈窗')
        except:
            print(f'執行緒 {threading.current_thread().name} 無詳情頁面彈窗或無法關閉')

        # 抓取鞋碼並轉換為數字
        size_status = {size_column_names[uk_size]: 0 for uk_size in uk_sizes}
        try:
            size_elements = WebDriverWait(driver, 15).until(
                EC.presence_of_all_elements_located((By.XPATH, '//div[contains(@class, "swatch-option text") and (contains(@data-option-label, "UK") or contains(@data-option-label, "US"))]'))
            )
            if size_elements:
                for size in size_elements:
                    size_label = size.get_attribute("data-option-label")
                    if not size_label:
                        continue
                    uk_size = None
                    if "UK" in size_label:
                        uk_size = size_label.replace("UK", "").strip()
                    elif "US" in size_label:
                        us_size = size_label.replace("US", "").strip()
                        uk_size = us_to_uk.get(us_size)
                    if uk_size and uk_size in uk_sizes:
                        style = size.get_attribute("style")
                        status = 2 if style == "" else 1  # 有貨: 2, 缺貨: 1
                        size_status[size_column_names[uk_size]] = status
                        print(f'執行緒 {threading.current_thread().name} 鞋碼 UK{uk_size}: {status}')
            else:
                print(f'執行緒 {threading.current_thread().name} 產品 {global_index+1} 無鞋碼')
        except:
            print(f'執行緒 {threading.current_thread().name} 產品 {global_index+1} 抓取失敗：鞋碼元素未找到')
            size_status = {size_column_names[uk_size]: 0 for uk_size in uk_sizes}

        # 抓取 SKU
        try:
            sku_element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//div[@class="value" and @itemprop="sku"]'))
            )
            sku = sku_element.text.strip()
            print(f'執行緒 {threading.current_thread().name} 產品 {global_index+1} SKU: {sku}')
        except:
            sku = "無 SKU"
            print(f'執行緒 {threading.current_thread().name} 產品 {global_index+1} 無 SKU')

        with lock:
            size_status_list[index] = size_status
            sku_list[index] = sku

    except Exception as e:
        print(f'執行緒 {threading.current_thread().name} 產品 {global_index+1} 抓取失敗: {e}')
        with lock:
            size_status_list[index] = {size_column_names[uk_size]: 0 for uk_size in uk_sizes}
            sku_list[index] = "抓取失敗"
    finally:
        driver.quit()

# 主程式
driver = webdriver.Chrome()
url = 'https://marathonsports.hkstore.com/marathon_en_us/women/footwear'
try:
    driver.get(url)
    time.sleep(5)
except:
    print(f'failed to open {url}')

try:
    cookie_button = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "Accept") or contains(text(), "Agree") or contains(text(), "OK")]'))
    )
    driver.execute_script("arguments[0].click();", cookie_button)
    print("已關閉彈窗")
except:
    print("無彈窗或無法關閉")

# 初始化清單儲存所有頁面的資料
all_brand_list = []
all_product_name_list = []
all_gender_list = []
all_type_list = []
all_price_list = []
all_discount1_list = []
all_discount2_list = []
all_discount3_list = []
all_discount1_weighted_list = []
all_discount2_weighted_list = []
all_discount3_weighted_list = []
all_href_list = []
all_size_status_list = []
all_sku_list = []

# 用於記錄全局產品序號（用於 scrape_id 和日誌）
global_product_index = 0

# 分頁處理，限制前 20 頁
current_page = 1
while current_page <= 20:
    print(f"\n正在處理第 {current_page} 頁...\n")
    
    # 初始化當前頁面的資料清單
    brand_list = []
    product_name_list = []
    gender_list = []
    type_list = []
    price_list = []
    discount1_list = []
    discount2_list = []
    discount3_list = []
    discount1_weighted_list = []
    discount2_weighted_list = []
    discount3_weighted_list = []
    href_list = []
    
    # 滾動頁面以載入所有產品
    previous_count = 0
    max_attempts = 5
    attempt = 0
    while attempt < max_attempts:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)
        result_list = driver.find_elements(By.XPATH, '//div[@class="product-item-info"]')
        current_count = len(result_list)
        print(f'第 {current_page} 頁滾動後找到 {current_count} 個產品')
        if current_count == previous_count:
            break
        previous_count = current_count
        attempt += 1

    # 抓取當前頁面的品牌
    brand_elements = driver.find_elements(By.XPATH, '//div[@class="product-item-brand"]')
    print(f'第 {current_page} 頁找到 {len(brand_elements)} 個品牌')
    for brand in brand_elements:
        print(brand.text)
        brand_list.append(brand.text)

    # 抓取當前頁面的產品名稱
    product_name_elements = driver.find_elements(By.XPATH, '//strong[@class="product name product-item-name"]')
    print(f'第 {current_page} 頁找到 {len(product_name_elements)} 個產品名稱')
    for name in product_name_elements:
        print(name.text)
        product_name_list.append(name.text)

    # 抓取當前頁面的 Gender 和 Type
    gender_type_elements = driver.find_elements(By.XPATH, '//div[@class="product-item-gender-type"]')
    print(f'第 {current_page} 頁找到 {len(gender_type_elements)} 個性別類型')
    for gender_type in gender_type_elements:
        text = gender_type.text.strip()
        print(f'原始 GenderType: {text}')
        if text:
            parts = text.split(" ")
            gender = parts[0]
            type_ = " ".join(parts[1:]) if len(parts) > 1 else "未知類型"
        else:
            gender = "未知性別"
            type_ = "未知類型"
        print(f'Gender: {gender}, Type: {type_}')
        gender_list.append(gender)
        type_list.append(type_)

    # 抓取當前頁面的最終價格
    price_elements = driver.find_elements(By.XPATH, '//span[@data-price-type="finalPrice"]//span[@class="price"]')
    print(f'第 {current_page} 頁找到 {len(price_elements)} 個最終價格')
    for price in price_elements:
        print(price.text)
        price_text = price.text.replace("HK$", "").replace(",", "")
        try:
            price_value = float(price_text)
        except ValueError:
            price_value = 0.0
        price_list.append(price_value)

    # 抓取當前頁面的折扣並拆分
    for item in result_list:
        left_discount = item.find_elements(By.XPATH, './/div[contains(@class, "cataloglabel-bottom-left")]//span[contains(text(), "Off") or contains(text(), "Buy")]')
        right_discount = item.find_elements(By.XPATH, './/div[contains(@class, "cataloglabel-botton-right")]//span[contains(text(), "Off") or contains(text(), "Buy")]')
        
        # 抓取左折扣
        left_text = left_discount[0].text.strip() if left_discount else ""
        left_discounts = []
        if left_text and left_text not in ["無左折扣", ""]:
            left_discounts = [d.strip() for d in left_text.split("|")]
        
        # 抓取右折扣
        right_text = right_discount[0].text.strip() if right_discount else ""
        right_discounts = []
        if right_text and right_text not in ["無右折扣", ""]:
            right_discounts = [d.strip() for d in right_text.split("|")]
        
        # 合併所有有效的折扣
        all_discounts = left_discounts + right_discounts
        if not all_discounts:
            all_discounts = ["No Discount"]
        all_discounts = all_discounts[:3]
        while len(all_discounts) < 3:
            all_discounts.append("No Discount")
        
        discount1_list.append(all_discounts[0])
        discount2_list.append(all_discounts[1])
        discount3_list.append(all_discounts[2])
        
        discount1_weighted_list.append(calculate_discount_weight(all_discounts[0]))
        discount2_weighted_list.append(calculate_discount_weight(all_discounts[1]))
        discount3_weighted_list.append(calculate_discount_weight(all_discounts[2]))
        
        print(f'折扣1: {all_discounts[0]} (加權值: {discount1_weighted_list[-1]}) | 折扣2: {all_discounts[1]} (加權值: {discount2_weighted_list[-1]}) | 折扣3: {all_discounts[2]} (加權值: {discount3_weighted_list[-1]})')

    # 抓取當前頁面的 href
    for item in result_list:
        try:
            link = item.find_element(By.XPATH, './/a[.//img[@class="product-image-photo"]]')
            href = link.get_attribute("href")
            href_list.append(href)
        except:
            href_list.append("無連結")

    # 初始化當前頁面的鞋碼狀態和 SKU 清單
    total_products_in_page = len(product_name_list)
    size_status_list = [None] * total_products_in_page
    sku_list = [None] * total_products_in_page

    # 使用多執行緒抓取當前頁面每雙鞋的內頁資料
    max_workers = 4
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = []
        for index in range(total_products_in_page):
            if href_list[index] == "無連結":
                print(f'第 {current_page} 頁產品 {index+1} 無連結，跳過')
                size_status_list[index] = {size_column_names[uk_size]: 0 for uk_size in uk_sizes}
                sku_list[index] = "無連結"
                continue
            future = executor.submit(fetch_details, index, product_name_list[index], href_list[index], size_status_list, sku_list, global_product_index)
            futures.append(future)
            global_product_index += 1  # 更新全局產品序號

        for future in futures:
            future.result()

    # 將當前頁面的資料追加到總清單中
    all_brand_list.extend(brand_list)
    all_product_name_list.extend(product_name_list)
    all_gender_list.extend(gender_list)
    all_type_list.extend(type_list)
    all_price_list.extend(price_list)
    all_discount1_list.extend(discount1_list)
    all_discount2_list.extend(discount2_list)
    all_discount3_list.extend(discount3_list)
    all_discount1_weighted_list.extend(discount1_weighted_list)
    all_discount2_weighted_list.extend(discount2_weighted_list)
    all_discount3_weighted_list.extend(discount3_weighted_list)
    all_href_list.extend(href_list)
    all_size_status_list.extend(size_status_list)
    all_sku_list.extend(sku_list)

    # 如果已經到達第 20 頁，停止爬取
    if current_page == 20:
        print("已到達第 20 頁，停止爬取")
        break

    # 檢查是否有下一頁
    try:
        # 嘗試找到「下一頁」按鈕（> 符號）
        next_page_button = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, '//a[contains(@class, "page") and .//span[contains(text(), ">")]]'))
        )
        print(f"找到下一頁按鈕，準備進入第 {current_page + 1} 頁")
        driver.execute_script("arguments[0].click();", next_page_button)
        time.sleep(5)  # 等待下一頁載入
        current_page += 1
    except:
        print("沒有找到下一頁按鈕，抓取完成")
        break

# 關閉主頁面的瀏覽器
driver.quit()

# 調整清單長度（確保所有欄位行數一致）
max_length = max(len(all_brand_list), len(all_product_name_list), len(all_gender_list), len(all_type_list), len(all_price_list), len(all_discount1_list), len(all_discount2_list), len(all_discount3_list), len(all_sku_list))
for lst in [all_brand_list, all_product_name_list, all_gender_list, all_type_list, all_price_list, all_discount1_list, all_discount2_list, all_discount3_list, all_discount1_weighted_list, all_discount2_weighted_list, all_discount3_weighted_list, all_sku_list, all_size_status_list]:
    while len(lst) < max_length:
        lst.append("N/A")

# 根據 url 判斷 website 的值
if url.startswith('https://marathonsports.hkstore.com/marathon_en_us/'):
    website_value = 0
    website_code = "01"  # 馬拉松
elif url.startswith('https://catalog.hkstore.com/catalog_en_us/'):
    website_value = 1
    website_code = "02"  # Catalog
else:
    website_value = 2
    website_code = "00"  # 其他網站

# 生成 scrape_id（格式：網站 + 日期 + 編號）
scrape_date = datetime.now().strftime("%y%m%d")  # 日期格式為 YYMMDD，例如 250414
scrape_ids = [f"{website_code}{scrape_date}{str(i).zfill(2)}" for i in range(len(all_product_name_list))]

# 整理成表格
data = {
    'scrape_id': scrape_ids,
    'shoe_brand': all_brand_list,
    'shoe_name': all_product_name_list,
    'shoe_gender': all_gender_list,
    'shoe_type': all_type_list,
    'price': all_price_list,
    'discount_1_text': all_discount1_list,
    'discount_1_weighted': all_discount1_weighted_list,
    'discount_2_text': all_discount2_list,
    'discount_2_weighted': all_discount2_weighted_list,
    'discount_3_text': all_discount3_list,
    'discount_3_weighted': all_discount3_weighted_list,
    'shoe_sku': all_sku_list,
    'website': [website_value] * len(all_product_name_list),
    'page_ranking': list(range(len(all_product_name_list)))
}

# 將鞋碼狀態加入表格
for uk_size in uk_sizes:
    column_name = size_column_names[uk_size]
    data[column_name] = [status[column_name] for status in all_size_status_list]

df = pd.DataFrame(data)

# 添加抓取日期
scrape_date_full = datetime.now().strftime("%Y-%m-%d")
df['scrape_time'] = scrape_date_full

# 分成兩個表格並輸出為 CSV
date_prefix = datetime.now().strftime("%Y%m%d")  # 格式為 YYYYMMDD，例如 20250414

# 第一個表格：marathon_shoes.csv
df_shoes = df[['shoe_sku', 'shoe_name', 'shoe_brand', 'shoe_type', 'shoe_gender']]
df_shoes.to_csv(f'{date_prefix}_marathon_shoes.csv', index=False, encoding='utf-8-sig')
print(f"已將第一個表格儲存為 '{date_prefix}_marathon_shoes.csv'")

# 第二個表格：marathon_scrape_data.csv
df_scrape_data = df[['scrape_id', 'shoe_sku', 'price', 
                     'discount_1_text', 'discount_1_weighted', 
                     'discount_2_text', 'discount_2_weighted', 
                     'discount_3_text', 'discount_3_weighted',
                     'uk_size_5', 'uk_size_5_5', 'uk_size_6', 'uk_size_6_5', 'uk_size_7',
                     'uk_size_7_5', 'uk_size_8', 'uk_size_8_5', 'uk_size_9', 'uk_size_9_5', 'uk_size_10',
                     'page_ranking', 'website', 'scrape_time']]
df_scrape_data.to_csv(f'{date_prefix}_marathon_scrape_data.csv', index=False, encoding='utf-8-sig')
print(f"已將第二個表格儲存為 '{date_prefix}_marathon_scrape_data.csv'")

# 顯示兩個表格
print("\n第一個表格 (marathon_shoes):")
display(df_shoes)
print("\n第二個表格 (marathon_scrape_data):")
display(df_scrape_data)

無彈窗或無法關閉

正在處理第 1 頁...

第 1 頁滾動後找到 24 個產品
第 1 頁滾動後找到 24 個產品
第 1 頁找到 24 個品牌
Hoka
Hoka
Hoka
Hoka
Hoka
Hoka
Hoka
Hoka
Hoka
Hoka
Hoka
Hoka
Hoka
Hoka
Hoka
Hoka
Hoka
Hoka
Hoka
Hoka
Hoka
Hoka
Merrell
Hoka
第 1 頁找到 24 個產品名稱
Arahi 7
Mach 6 LA
MACH 6
SKYFLOW
Mach 6
Speedgoat 6 GORE-TEX
Ora Primo
Kaha 2 Frost Moc GORE-TEX
Speedgoat 6
Stinson EVO OG
Restore TC
Clifton One9
Bondi 9
Project Transport
Transport GORE-TEX
Stinson EVO OG
Tor Summit
Mach 6 LA
Hopara
Bondi 8
Clifton 9
Anacapa 2 Low GORE-TEX
Hydro Moc
Bondi 8
第 1 頁找到 24 個性別類型
原始 GenderType: Women Running Shoes
Gender: Women, Type: Running Shoes
原始 GenderType: Women Running Shoes
Gender: Women, Type: Running Shoes
原始 GenderType: Women Running Shoes
Gender: Women, Type: Running Shoes
原始 GenderType: Women Running Shoes
Gender: Women, Type: Running Shoes
原始 GenderType: Women Running Shoes
Gender: Women, Type: Running Shoes
原始 GenderType: Women Hiking Shoes
Gender: Women, Type: Hiking Shoes
原始 GenderType: Unisex Sneakers
Gender: Unisex, Type: Sn

,shoe_sku,shoe_name,shoe_brand,shoe_type,shoe_gender
0,1HK1147851BSW,Arahi 7,Hoka,Running Shoes,Women
1,1HK1164033BBLC,Mach 6 LA,Hoka,Running Shoes,Women
2,1HK1147810WTLT,MACH 6,Hoka,Running Shoes,Women
3,1HK1155113FSTS,SKYFLOW,Hoka,Running Shoes,Women
4,1HK1147810BKFS,Mach 6,Hoka,Running Shoes,Women
5,1HK1155151SZQ,Speedgoat 6 GORE-TEX,Hoka,Hiking Shoes,Women
6,1HK1141570ORZ,Ora Primo,Hoka,Sneakers,Unisex
7,1HK1155196OKS,Kaha 2 Frost Moc GORE-TEX,Hoka,Sneakers,Women
8,1HK1147811OMN,Speedgoat 6,Hoka,Hiking Shoes,Women
9,1HK1155350BLTR,Stinson EVO OG,Hoka,Trail Running Shoes,Unisex



第二個表格 (marathon_scrape_data):


,scrape_id,shoe_sku,price,discount_1_text,discount_1_weighted,discount_2_text,discount_2_weighted,discount_3_text,discount_3_weighted,uk_size_5,...,uk_size_7,uk_size_7_5,uk_size_8,uk_size_8_5,uk_size_9,uk_size_9_5,uk_size_10,page_ranking,website,scrape_time
0,0125041400,1HK1147851BSW,839.0,No Discount,100,No Discount,100,No Discount,100,2,...,1,1,1,0,0,0,0,0,0,2025-04-14
1,0125041401,1HK1164033BBLC,879.0,No Discount,100,No Discount,100,No Discount,100,1,...,2,0,0,0,0,0,0,1,0,2025-04-14
2,0125041402,1HK1147810WTLT,1199.0,10% Off,90,No Discount,100,No Discount,100,2,...,2,2,0,0,0,0,0,2,0,2025-04-14
3,0125041403,1HK1155113FSTS,1349.0,10% Off,90,No Discount,100,No Discount,100,2,...,2,0,0,0,0,0,0,3,0,2025-04-14
4,0125041404,1HK1147810BKFS,1199.0,10% Off,90,No Discount,100,No Discount,100,2,...,1,2,0,0,0,0,0,4,0,2025-04-14
5,0125041405,1HK1155151SZQ,1599.0,10% Off,90,No Discount,100,No Discount,100,2,...,1,2,2,0,0,0,0,5,0,2025-04-14
6,0125041406,1HK1141570ORZ,999.0,No Discount,100,No Discount,100,No Discount,100,1,...,2,2,1,2,0,1,0,6,0,2025-04-14
7,0125041407,1HK1155196OKS,1899.0,10% Off,90,No Discount,100,No Discount,100,2,...,1,1,1,0,0,0,0,7,0,2025-04-14
8,0125041408,1HK1147811OMN,1299.0,10% Off,90,No Discount,100,No Discount,100,2,...,2,2,2,0,0,0,0,8,0,2025-04-14
9,0125041409,1HK1155350BLTR,1499.0,10% Off,90,No Discount,100,No Discount,100,0,...,2,2,1,2,2,2,2,9,0,2025-04-14
